In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import random

In [2]:
def LinearRegressor(table, random):
    X = table.iloc[:, :-1]
    y = table.iloc[:, -1]
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.35, random_state = random)
    
    model = LinearRegression()
    model.fit(x_train,y_train)
    r_sq = model.score(x_test, y_test)
    
    y_pred = model.predict(X)
    
# #     print('intercept:', model.intercept_)
   # print('R2_Score:', r_sq)
# #    print('slope:', model.coef_)  
    return model, model.coef_, r_sq

In [3]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(float)

In [4]:
def SquaredTable(table):
    for col in table.columns:
        if type(table[col].iloc[0]) != str:
            table[str(col) + ' ** 2'] = table[col] * table[col]
    
    return table

In [5]:
AllRounder_Table = pd.read_excel('2013-2018_AllRounder_table.xlsx')

In [7]:
AllRounder_Table.drop(AllRounder_Table.columns[:5], axis = 1, inplace = True)

In [9]:
AllRounder_Table = clean_dataset(AllRounder_Table)

In [11]:
SquaredTable(AllRounder_Table).to_excel('Squared_AllRounder.xlsx')

In [14]:
AllRounder_Table = pd.read_excel('Squared_AllRounder.xlsx')

In [16]:
def OutputR2Score(table):
    table = clean_dataset(table)
    if len(table) == 0:
        print("TABLE HAS NO ROWS! ERROR")
    random_state = findMaxRState(table)
    r_sq = LinearRegressor(table, random_state)[2]
    
    dictionary = dict()
    for i in range(10, len(table.columns)):
        max_r2 = returnMaxR2Score(table, i)[2]
        dictionary[i] = max_r2
        
#     cols = returnMaxR2Score(table, max(dictionary, key=dictionary.get))[1]
#     return table[cols], cols, random_state
# #     print(dictionary)
    return dictionary[max(dictionary, key=dictionary.get)], returnMaxR2Score(table, max(dictionary, key=dictionary.get))[1], max(dictionary, key=dictionary.get), dictionary

In [17]:
def findMaxRState(table):
    maxim = dict()
    for count in range(0,1000):
        r_score = LinearRegressor(table, count)[2]
        maxim[count] = r_score
    return max(maxim, key=maxim.get)

In [18]:
def tryoutColumns(table, num):
    arr = []
    while len(arr) < num:
        gen = random.randint(0, len(table.columns)-2)
        if gen not in arr:
            arr.append(gen)
    
    names = []
    for elm in arr:
        names.append(table.columns[elm])
    return names

In [19]:
def returnMaxR2Score(table, num):
    cols = tryoutColumns(table, num)
    cols.append('IPL Pts per Match Year+2')
    bats = table[cols]
    r2_score = LinearRegressor(bats, findMaxRState(bats))[2]
    return bats, cols, r2_score
    

In [24]:
OutputR2Score(AllRounder_Table)

(0.31627981580420783,
 ['Year+1 Bowling Innings',
  'Year+1 Bowling SR in Powerplay Overs ** 2',
  'Year 4w',
  'Year Bowling Econ. Rate in Powerplay Overs ** 2',
  'Year+1 Bowling Econ. Rate T20I',
  'Year Runs',
  'Year+1 Highest Score',
  'Year Runs Conceded ** 2',
  'Year Total 6s ** 2',
  'Year Batting Avg PowerPlay ** 2',
  'Year+1 Batting Avg PowerPlay',
  'Year+1 Bowling Econ. Rate in Death Overs ** 2',
  'Year Bowling Average in Death Overs',
  'Year+1 Batting Avg',
  'Year Age ** 2',
  'Year Bowling Econ. Rate in Powerplay Overs',
  'Year+1 Total 50s',
  'Year+1 Bowling Econ. Rate T20I ** 2',
  'Year+1 Bowling SR T20I',
  'Year+1 Total 100s',
  'IPL Pts per Match Year+2'],
 20,
 {10: 0.12218393755605082,
  11: 0.26154877854705294,
  12: 0.14151947323281655,
  13: -0.03560669411742334,
  14: 0.08906169183645762,
  15: 0.29564464216487374,
  16: 0.2577187534262674,
  17: 0.19922534471425613,
  18: 0.08156674034949551,
  19: 0.09449491279163913,
  20: 0.31627981580420783,
  21: 

In [26]:
a = returnMaxR2Score(AllRounder_Table, 20)
while a[2] < 0.38:
    a = returnMaxR2Score(AllRounder_Table, 20)

In [28]:
LinearRegressor(a[0], findMaxRState(a[0]))

(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 array([-1.08760243e-01, -2.36725475e-05, -2.07977280e-01,  6.59323996e-02,
         1.97975593e-04,  1.33866637e-02,  1.38655024e-05,  5.33359933e-03,
        -1.51484335e-01,  3.77724448e-02,  1.87219847e-03,  3.27133984e-03,
         5.80949027e-04,  2.20911750e+00, -3.47446171e-02,  2.47724140e+00,
         5.57699944e-03,  8.36117124e-02, -4.67608452e-02,  4.87876631e-04]),
 0.39708441151643115)

In [29]:
random2 = findMaxRState(a[0])
print(random2)
LinearRegressor(a[0], random2)

828


(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 array([-1.08760243e-01, -2.36725475e-05, -2.07977280e-01,  6.59323996e-02,
         1.97975593e-04,  1.33866637e-02,  1.38655024e-05,  5.33359933e-03,
        -1.51484335e-01,  3.77724448e-02,  1.87219847e-03,  3.27133984e-03,
         5.80949027e-04,  2.20911750e+00, -3.47446171e-02,  2.47724140e+00,
         5.57699944e-03,  8.36117124e-02, -4.67608452e-02,  4.87876631e-04]),
 0.39708441151643115)

In [30]:
from sklearn.model_selection import KFold, cross_val_score

In [43]:
def crossValidation(model, X, y, split_num):
    k_fold = KFold(n_splits=split_num)
    for train_indices, test_indices in k_fold.split(X):
        print('Train: %s | test: %s' % (train_indices, test_indices))
    return cross_val_score(model, X, y, cv=k_fold, n_jobs = -1)

In [34]:
def pretty_print_coefs(intercept, coefs, columns):
    lst = zip(coefs, columns)
    return str(intercept) + ' ' + " + ".join("%s * %s" % (round(coef, 3), name)
                                   for coef, name in lst)

In [35]:
X,y = a[0].iloc[:,:-1], a[0].iloc[:,-1]
mod = LinearRegressor(a[0], findMaxRState(a[0]))[0]

In [40]:
crossvalmean= dict()
for i in range(3,30):
    crossvalmean[i] = crossValidation(mod, X,y,i).max()

In [48]:
cros = crossValidation(mod,X,y,18)
cros

Train: [  7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24
  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42
  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60
  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78
  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96
  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112] | test: [0 1 2 3 4 5 6]
Train: [  0   1   2   3   4   5   6  14  15  16  17  18  19  20  21  22  23  24
  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42
  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60
  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78
  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96
  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112] | test: [ 7  8  9 10 11 12 13]
Train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  21  

array([-1.56170563e-01,  1.14788497e-02, -7.73342028e-01, -8.25774801e-01,
       -1.54877508e+00, -1.82866696e+01,  4.55539027e-01, -4.10968714e+00,
       -5.85983486e-01, -1.36205232e-01, -1.45227661e+00, -7.66031412e-01,
       -1.95730831e+00, -8.53151101e+00, -4.84007450e+05, -2.54913268e-01,
       -5.23209829e-01, -6.91631412e-01])

In [49]:
cros[6]

0.45553902673353935

In [50]:
train_indices = pd.Series(np.arange(0,41)).append(pd.Series(np.arange(47,113)))
test_indices = np.arange(41,47)

In [51]:
train_indices

0       0
1       1
2       2
3       3
4       4
     ... 
61    108
62    109
63    110
64    111
65    112
Length: 107, dtype: int64

In [52]:
mod.fit(X.iloc[train_indices], y.iloc[train_indices])
mod.score(X.iloc[test_indices], y.iloc[test_indices])

0.4555390267335274

In [53]:
pretty_print_coefs(mod.intercept_, mod.coef_, X.columns)

'12.072582586968323 -0.092 * Year+1 Matches + 0.0 * Year Bowling Average T20I ** 2 + -0.167 * Year+1 Total 50s ** 2 + 0.064 * Year Bowling SR in Death Overs + 0.0 * Year Overs ** 2 + 0.015 * Year Batting Avg PowerPlay + 0.0 * Year Runs ** 2 + 0.015 * Year Batting Strike Rate PowerPlay + -0.148 * Year Age + 0.001 * Year Bowling Econ. Rate in Death Overs + 0.004 * Year Runs Conceded + -0.008 * Year+1 Bowling Average + 0.0 * Year+1 Bowling SR T20I ** 2 + 1.394 * Year+1 5w ** 2 + -0.049 * Year Bowling Strike Rate + 2.991 * Year+1 Intl. + -0.001 * Year Bowling Innings ** 2 + 0.011 * Year+1 Innings + -0.005 * Year Innings + 0.001 * Year+1 Bowling Econ. Rate in Death Overs ** 2'

In [54]:
X

,Year+1 Matches,Year Bowling Average T20I ** 2,Year+1 Total 50s ** 2,Year Bowling SR in Death Overs,Year Overs ** 2,Year Batting Avg PowerPlay,Year Runs ** 2,Year Batting Strike Rate PowerPlay,Year Age,Year Bowling Econ. Rate in Death Overs,Year Runs Conceded,Year+1 Bowling Average,Year+1 Bowling SR T20I ** 2,Year+1 5w ** 2,Year Bowling Strike Rate,Year+1 Intl.,Year Bowling Innings ** 2,Year+1 Innings,Year Innings,Year+1 Bowling Econ. Rate in Death Overs ** 2
0,7,928.324681,0.000000,12.601727,576.00,26.491565,2116.0,104.842653,31,10.110204,159.0,31.500000,435.369810,0.0,20.57,1,49.0,5.000000,3.0,102.304796
1,10,928.324681,1.184256,8.700000,3844.00,1.000000,7921.0,20.000000,30,8.600000,394.0,48.570000,144.000000,0.0,17.71,0,289.0,9.161765,8.0,153.760000
2,14,136.890000,1.184256,21.000000,3249.00,26.491565,2704.0,104.842653,22,8.200000,371.0,17.700000,547.560000,0.0,26.30,0,256.0,9.161765,9.0,65.610000
3,9,928.324681,0.000000,6.333333,16.00,47.500000,501264.0,129.550000,33,10.600000,40.0,38.778209,36.000000,0.0,8.00,1,16.0,9.000000,16.0,102.304796
4,10,2601.000000,0.000000,7.000000,492.84,8.000000,30276.0,133.330000,29,11.600000,168.0,52.330000,400.000000,0.0,16.75,1,81.0,9.000000,9.0,210.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,8,408.040000,0.000000,9.250000,3169.69,3.500000,32041.0,58.330000,34,10.100000,485.0,49.000000,343.879936,0.0,16.95,1,256.0,7.000000,15.0,110.250000
109,8,1024.000000,0.000000,13.579766,289.00,1.000000,3969.0,50.000000,31,11.007273,134.0,12.500000,343.879936,0.0,102.00,0,121.0,7.000000,6.0,113.978833
110,1,648.409637,1.247428,13.579766,81.00,24.302424,169.0,93.485455,31,11.007273,64.0,15.500000,343.879936,0.0,18.00,0,9.0,8.870130,2.0,113.978833
111,15,648.409637,1.000000,13.579766,36.00,16.333333,130321.0,181.480000,33,11.007273,33.0,41.000000,343.879936,0.0,36.00,0,16.0,11.000000,13.0,113.978833


In [56]:
a[0].to_excel('All-RounderFinal.xlsx')

In [67]:
y_pred = mod.predict(X)
InputOutputAllRounder = pd.DataFrame(columns = ['Player Name','Season Year+2','IPL Pts per Match Year+2', 'Predicted Points', 'Actual Salary Year+2'])
InputOutputAllRounder['IPL Pts per Match Year+2'] = y
InputOutputAllRounder['Predicted Points'] = y_pred

In [69]:
AllRounder_Table2 = pd.read_excel('2013-2018_AllRounder_table.xlsx')

In [70]:
for i in range(len(InputOutputAllRounder)):
    for count in range(len(AllRounder_Table2)):
        if InputOutputAllRounder['IPL Pts per Match Year+2'][i] == AllRounder_Table2['IPL Pts per Match Year+2'][count]:
            if X['Year Bowling SR in Death Overs'][i] == AllRounder_Table2['Year Bowling SR in Death Overs'][count]:
                if X['Year Bowling Econ. Rate in Death Overs'][i] == AllRounder_Table2['Year Bowling Econ. Rate in Death Overs'][count]:
                    if X['Year Batting Avg PowerPlay'][i] == AllRounder_Table2['Year Batting Avg PowerPlay'][count]:
                        InputOutputAllRounder['Player Name'][i] = AllRounder_Table2['Player Name'][count]
                        InputOutputAllRounder['Season Year+2'][i] = AllRounder_Table2['Year+1 Season'][count]+1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [71]:
def ActualSalary(Season, player):
    master = pd.read_excel(str(Season)+'FinalMasterSheet.xlsx', header=1)
    for count in range(len(master)):
        if master['Player Name'][count] == str(player):
            return master['Salary(Rupees Crore)'][count]

In [72]:
for count in range(len(InputOutputAllRounder)):
    if InputOutputAllRounder['Season Year+2'][count] != 2020:
        InputOutputAllRounder['Actual Salary Year+2'][count] = ActualSalary(InputOutputAllRounder['Season Year+2'][count],InputOutputAllRounder['Player Name'][count] )
    else:
        InputOutputAllRounder['Actual Salary Year+2'][count] = 'None'

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
InputOutputAllRounder

,Player Name,Season Year+2,IPL Pts per Match Year+2,Predicted Points,Actual Salary Year+2
0,Albie Morkel,2015,16.125000,12.389508,0.3
1,A Mishra,2015,9.291667,9.251563,3.5
2,B Kumar,2015,15.000000,12.011197,4.25
3,CH Gayle,2015,16.857143,16.731581,8.4
4,DJG Sammy,2015,2.250000,12.537005,2.8
...,...,...,...,...,...
108,SR Watson,2018,21.200000,13.016609,4
109,STR Binny,2018,2.357143,4.046719,0.5
110,Vinay Kumar,2018,11.250000,9.330446,1
111,YK Pathan,2018,7.100000,8.949790,1.9


In [74]:
InputOutputAllRounder.to_excel('InputOutputAllRounder.xlsx')